# Question Answering Over Documents

_Augmented with document retrieval from Vertex AI Search and LangChain_

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fretrieval-augmented-generation%2Fexamples%2Fquestion_answering.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/retrieval-augmented-generation/examples/question_answering.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


---


* Authors: Ruchika Kharwar, Tomas Paquete, Tom Pakeman, Holt Skinner
* Created: 06/06/2023

---

## Objective

This notebook shows how to ask and answer questions about your data by combining a Vertex AI Search engine with LLMs. In particular, we focus on querying 'unstructured' data such as PDFs and HTML files.

These patterns are useful if you have a Vertex AI Search Engine pointed at a store of documents, such as a Google Cloud Storage bucket containing PDFs.

In order to run this notebook you must have created an unstructured search engine and ingested PDF or HTML documents into it.

Users may wish to:

1. Get an answer to a question from document(s)
2. Get an answer to a question from document(s) along with the citation/sources
3. Get an answer to a question from document(s) and follow it up with additional questions like in a conversation.
4. Users might want to change the default prompt of LangChain while using the results of Vertex AI Search

In each case we will use an example of the LangChain [retriever](https://python.langchain.com/v0.2/docs/concepts/#retrievers) interface to fetch contextual documents & segments from a Vertex AI Search engine, and the Vertex AI API to call Google's Gemini model for summarization & answering.

Please note that the prompts and examples demonstrated here will need to be customized in order to work well with your own data.

---

In this notebook the following examples will be elaborated:

- ✅ Example of using `VertexAISearchRetriever` with LangChain `RetrievalQA`
- ✅ Example of using `VertexAISearchRetriever` with LangChain `RetrievalQAWithSourcesChain`
- ✅ Example of using `VertexAISearchRetriever` with LangChain `ConversationalRetrievalChain`
- ✅ Example of using `VertexAIMultiTurnSearchRetriever` with LangChain `ConversationalRetrievalChain`
- ✅ Examples of changing the default prompt with Vertex AI Search and `RetrievalQAWithSourcesChain`

---

**References:**

- [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)
- [LangChain Python Documentation](https://python.langchain.com/en/latest/)
- [LangChain Vertex AI Search Retriever Documentation](https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search)


## High level flow

![Vertex_AI_Search_RAG](https://storage.googleapis.com/github-repo/search/examples/VAIS_RAG.png)

The following is a high-level overview of the steps in the examples that are going to be demonstrated:

- Users enter a prompt of a question
- The datastore is then queried and relevant documents are returned
- The documents are added as context into the user's prompt/question
- The prompt is sent into the LLM
- A response is returned using the relevant documents as sources


Services used in the notebook:

- ✅ Vertex AI Gemini API for LLM
- ✅ Vertex AI Search for document search and retrieval
- ✅ LangChain in order to chain tasks


## Install pre-requisites

If running in Colab install the pre-requisites into the runtime. Otherwise it is assumed that the notebook is running in Vertex AI Workbench. In that case it is recommended to install the pre-requisites from a terminal using the `--user` option.


In [ ]:
%pip install -q --upgrade --user google-cloud-aiplatform google-cloud-discoveryengine langchain langchain-google-vertexai langchain-google-community

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



## Authenticate

If running in Colab authenticate with `google.colab.google.auth` otherwise assume that running on Vertex AI Workbench.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

## Configure notebook environment

To get started using Vertex AI Search, you must have an existing Google Cloud project and [enable the Vertex AI API and Vertex AI Search API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,discoveryengine.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).


In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Set the following constants to reflect your environment

- The queries used in the examples here relate to a GCS bucket containing Alphabet investor PDFs, but these should be customized to your own data.


In [ ]:
DATA_STORE_ID = "YOUR_DATA_STORE_ID"  # @param {type:"string"}
DATA_STORE_LOCATION = "global"  # @param {type:"string"}

MODEL = "gemini-1.0-pro"  # @param {type:"string"}

if PROJECT_ID == "YOUR_PROJECT_ID" or DATA_STORE_ID == "YOUR_DATA_STORE_ID":
    raise ValueError(
        "Please set the PROJECT_ID, DATA_STORE_ID constants to reflect your environment."
    )

### Import libraries

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from langchain_google_vertexai import VertexAI
from langchain_google_community import VertexAISearchRetriever
from langchain_google_community import VertexAIMultiTurnSearchRetriever

## LangChain retrieval Q&A chains

We will demonstrate using three LangChain retrieval Q&A chains:

- `RetrievalQA`
- `RetrievalQAWithSourcesChain`
- `ConversationalRetrievalChain`

We begin by initializing a Vertex AI LLM and a LangChain 'retriever' to fetch documents from our Vertex AI Search engine.

For Q&A chains our retriever is passed directly to the chain and can be used automatically without any further configuration.

Behind the scenes, first the search query is passed to the retriever which runs a search and returns relevant document chunks. These chunks are then passed to the prompt used by the LLM to be used as context.


In [ ]:
llm = VertexAI(model_name=MODEL)

retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DATA_STORE_LOCATION,
    data_store_id=DATA_STORE_ID,
    get_extractive_answers=True,
    max_documents=10,
    max_extractive_segment_count=1,
    max_extractive_answer_count=5,
)

### [`RetrievalQA` chain](https://python.langchain.com/docs/modules/chains/popular/vector_db_qa)

This is the simplest document Q&A chain offered by LangChain.

There are several different chain types available, listed [here](https://docs.langchain.com/docs/components/chains/index_related_chains).

- In these examples we use the `stuff` type, which simply inserts all of the document chunks into the prompt.
- This has the advantage of only making a single LLM call, which is faster and more cost efficient
- However, if we have a large number of search results we run the risk of exceeding the token limit in our prompt, or truncating useful information.
- Other chain types such as `map_reduce` and `refine` use an iterative process which makes multiple LLM calls, taking individual document chunks at a time and refining the answer iteratively.


In [ ]:
search_query = "What was Alphabet's Revenue in Q2 2021?"  # @param {type:"string"}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)
retrieval_qa.invoke(search_query)

#### Inspecting the process

If we add `return_source_documents=True` we can inspect the document chunks that were returned by the retriever.

This is helpful for debugging, as these chunks may not always be relevant to the answer, or their relevance might not be obvious.


In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

results = retrieval_qa.invoke(search_query)

print("*" * 79)
print(results["result"])
print("*" * 79)
for doc in results["source_documents"]:
    print("-" * 79)
    print(doc.page_content)

### `RetrievalQAWithSourcesChain`

This variant returns an answer to the question alongside the source documents that were used to generate the answer.


In [ ]:
retrieval_qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

retrieval_qa_with_sources.invoke(search_query, return_only_outputs=True)

### [`ConversationalRetrievalChain`](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) with `VertexAIMultiTurnSearchRetriever`

`ConversationalRetrievalChain` remembers and uses previous questions so you can have a chat-like discovery process.

To use this chain we must provide a memory class to store and pass the previous messages to the LLM as context. Here we use the `ConversationBufferMemory` class that comes with LangChain.

`VertexAIMultiTurnSearchRetriever` uses [multi-turn search](https://cloud.google.com/generative-ai-app-builder/docs/multi-turn-search) (also called conversational search or search with followups) to preserve context between requests.

The following example will work with both retrievers, and the multi-turn retriever can be substituted in any of the previous examples.

In [ ]:
multi_turn_retriever = VertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=DATA_STORE_LOCATION, data_store_id=DATA_STORE_ID
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=multi_turn_retriever, memory=memory
)

search_query = "What were alphabet revenues in 2022?"

result = conversational_retrieval.invoke(search_query)
print(result["answer"])

In [ ]:
new_query = "What about costs and expenses?"
result = conversational_retrieval.invoke(new_query)
print(result["answer"])

In [ ]:
new_query = "Is this more than in 2021?"

result = conversational_retrieval.invoke(new_query)
print(result["answer"])

## Advanced: Modifying the default langchain prompt

In all of the previous examples we used the default prompt that comes with langchain.

We can inspect our chain object to discover the wording of the prompt template being used.

We may find that this is not suitable for our purposes, and we may wish to customize the prompt, for example to present our results in a different format, or to specify additional constraints.


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

print(qa.combine_documents_chain.llm_chain.prompt.template)

---

Let's modify the prompt to return an answer in a single word (useful for yes/no questions). We will constrain the LLM to say 'I don't know' if it cannot answer.

We create a new prompt_template and pass this in using the `template` argument.


In [ ]:
prompt_template = """Use the context to answer the question at the end.
You must always use the context and context only to answer the question. Never try to make up an answer. If the context is empty or you do not know the answer, just say "I don't know".
The answer should consist of only 1 word and not a sentence.

Context: {context}

Question: {question}
Helpful Answer:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt, retriever=retriever, return_source_documents=True
)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

In [ ]:
search_query = "Were 2020 EMEA revenues higher than 2020 APAC revenues?"

results = qa_chain.invoke(search_query)

print("*" * 79)
print(results["result"])
print("*" * 79)
for doc in results["source_documents"]:
    print("-" * 79)
    print(doc.page_content)